This notebook will examine Helsinki region neighborhoods, their population and restaurants.
Population data is from Helsingi region infoshare: https://hri.fi/en_gb/ .
Restaurant info gathered using from Foursquare API.

The findings can be used by anyone looking to open a new restaurant in the area.


In [1]:
!pip install folium

    100% |████████████████████████████████| 92kB 7.5MB/s eta 0:00:01
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/

In [2]:
import folium

In [3]:
import types
import pandas as pd
# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about your possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_67e8d7d7f1b4488ba834556c53b1e69d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6YIJVlf6WlE5MhLIb-YEuGXvhmcp_8H1634yARH3ilkt',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

# Your data fil<e was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about your possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_67e8d7d7f1b4488ba834556c53b1e69d.get_object(Bucket='dscapstone-donotdelete-pr-h7kvwfaqi9kqkn', Key='PKS_aluejako_alueet_ja_pisteet.kml')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 
body = client_67e8d7d7f1b4488ba834556c53b1e69d.get_object(Bucket='dscapstone-donotdelete-pr-h7kvwfaqi9kqkn',Key='popul2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

streaming_body_2 = client_67e8d7d7f1b4488ba834556c53b1e69d.get_object(Bucket='dscapstone-donotdelete-pr-h7kvwfaqi9kqkn', Key='areas.kml')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(streaming_body_2, "__iter__"): streaming_body_2.__iter__ = types.MethodType( __iter__, streaming_body_2 ) 

#This is for the population data
df_data_1 = pd.read_csv(body,sep=";",error_bad_lines=False)
df_data_1.head()



b'Skipping line 154: expected 31 fields, saw 33\nSkipping line 277: expected 31 fields, saw 33\nSkipping line 347: expected 31 fields, saw 33\n'


,name,Seutukarttakoodi/Map code,Age,2000,2001,2002,2003,2004,2005,2006,...,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
0,Kruununhaka,911101010,Yhteensä/Total,7071,6983,6778,6763,6695,6688,6684,...,7460,7470,7454,7434,7381,7325,7257,7180,7096,7026
1,Kluuvi,911101020,Yhteensä/Total,456,462,430,391,389,387,398,...,647,649,648,646,643,640,638,637,633,631
2,Katajanokka,911101080,Yhteensä/Total,4245,4220,4139,4161,4129,4167,4085,...,4578,4874,4995,4983,4937,4887,4831,4769,4701,4637
3,Kaartinkaupunki,911102030,Yhteensä/Total,1011,960,911,884,890,881,863,...,1048,1056,1062,1067,1069,1070,1067,1064,1059,1055
4,Punavuori,911102050,Yhteensä/Total,8041,8149,8273,8363,8378,8414,8218,...,9347,9417,9494,9459,9439,9467,9369,9257,9141,9048


In [4]:
# Location data for neigborhoods
data = streaming_body_2.read()

# Read greater Helsinki neighbourhood names and coordinates
# and store them into a pandas dataframe
import xml.etree.ElementTree as ET
import re
root = ET.fromstring(data)

neighlist = {'name': [], 'loc': [], 'lat': []}
   
for elem in root.iter("Placemark"):
    elem2 = elem.find("name")
    elem3 = elem.find("Point")
    if(elem3):
        coord = elem3.find("coordinates")
        #print(elem2.text, coord.text)  
        neighlist["name"].append(elem2.text)
        coords = re.split(",", coord.text)
        #print (coords[0])
        neighlist["loc"].append(coords[0])
        neighlist["lat"].append(coords[1])
        
 
df = pd.DataFrame(neighlist)
print(df.head())

                lat               loc         name
0  60.1651458720618  24.6084713225694   Saunaniemi
1  60.1766621010743  24.5901810696507     Kurttila
2  60.1852516841211  24.6166894947077     Vanttila
3  60.1421286563636   24.644213040441  Laurinlahti
4   60.128115882609  24.6695075699124  Soukanniemi


In [5]:
#Merge dataframe with the one that has income data
dfmerged = pd.merge(df_data_1,df, on=['name'])
dfmerged[["lat", "loc"]] = dfmerged[["lat", "loc"]].apply(pd.to_numeric)
dfmerged.drop_duplicates(subset="name", inplace=True)
#Just take columns we need, "2019" is the population figure for 2019
dfmerged=dfmerged[["name","lat","loc","2019"]]
dfmerged.head()

,name,lat,loc,2019
0,Kruununhaka,60.172643,24.964175,7470
1,Kluuvi,60.173882,24.942115,649
2,Katajanokka,60.166575,24.980714,4874
3,Kaartinkaupunki,60.165507,24.950692,1056
4,Punavuori,60.161397,24.935337,9417


In [6]:
# Function to show map and draw markers
def showMap(latitude, longitude, neighborhood_data):
    
    map = folium.Map(location=[latitude, longitude], zoom_start=10)

    # add markers to map
    for lat, lng, neighborhood in zip(neighborhood_data['lat'], neighborhood_data['loc'], neighborhood_data['name']):
        label = '{}'.format(neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map)  
    
    return map

In [7]:
# create map of Greater helsinki using latitude and longitude values
map = showMap(60.192059, 24.945831,dfmerged)
map

In [8]:
# Foursquare API credentials
CLIENT_ID = '' # removing the credentials for sharing on github
CLIENT_SECRET = '' 
VERSION = '20180605' # Foursquare API version

In [9]:
import requests

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
# Function to fetch venues within a given radius of a location
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
#Get venues for all the neighbourhoods
venues = getNearbyVenues(names=dfmerged['name'],
                                   latitudes=dfmerged['lat'],
                                   longitudes=dfmerged['loc']
                                  )

Kruununhaka
Kluuvi
Katajanokka
Kaartinkaupunki
Punavuori
Eira
Ullanlinna
Kaivopuisto
Suomenlinna
Länsisaaret
Kamppi
Etu-Töölö
Ruoholahti
Lapinlahti
Jätkäsaari
Taka-Töölö
Meilahti
Laakso
Vanha Munkkiniemi
Kuusisaari
Lehtisaari
Munkkivuori
Niemenmäki
Talinranta
Etelä-Haaga
Kivihaka
Pohjois-Haaga
Lassila
Konala
Pajamäki
Tali
Reimarla
Marttila
Kannelmäki
Maununneva
Malminkartano
Hakuninmaa
Kuninkaantammi
Siltasaari
Linjat
Torkkelinmäki
Harju
Alppila
Vallila
Länsi-Pasila
Pohjois-Pasila
Itä-Pasila
Keski-Pasila
Toukola
Arabianranta
Kumpula
Käpylä
Koskela
Vanhakaupunki
Pirkkola
Maunula
Metsälä
Maunulanpuisto
Länsi-Pakila
Paloheinä
Torpparinmäki
Haltiala
Patola
Veräjämäki
Veräjälaakso
Itä-Pakila
Tuomarinkartano
Viikinranta
Latokartano
Viikin tiedepuisto
Viikinmäki
Pihlajamäki
Pihlajisto
Pukinmäki
Ylä-Malmi
Ala-Malmi
Tattariharju
Tapaninvainio
Tapanila
Siltamäki
Töyrynummi
Tapulikaupunki
Puistola
Heikinlaakso
Tattarisuo
Jakomäki
Kulosaari
Länsi-Herttoniemi
Roihuvuori
Herttoniemenranta
Tammisalo


In [13]:
#How many venues we got
print(venues.shape)
venues.head()

(2833, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kruununhaka,60.172643,24.964175,Tervasaari / Tjärholmen,60.173535,24.967826,Park
1,Kruununhaka,60.172643,24.964175,Tervasaaren koira-aitaus,60.173111,24.967925,Dog Run
2,Kruununhaka,60.172643,24.964175,Ravintola Kolme Kruunua,60.174297,24.958925,Scandinavian Restaurant
3,Kruununhaka,60.172643,24.964175,Svenska Klubben,60.174144,24.959693,Event Space
4,Kruununhaka,60.172643,24.964175,Kuurna,60.170128,24.958564,Scandinavian Restaurant


In [14]:
# We only are interested in restaurants so let's drop others
venues = venues[venues['Venue Category'].str.contains("Restaurant")]
print(venues.shape)
venues.head()

(387, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,Kruununhaka,60.172643,24.964175,Ravintola Kolme Kruunua,60.174297,24.958925,Scandinavian Restaurant
4,Kruununhaka,60.172643,24.964175,Kuurna,60.170128,24.958564,Scandinavian Restaurant
5,Kruununhaka,60.172643,24.964175,Korea House,60.172910,24.956436,Korean Restaurant
8,Kruununhaka,60.172643,24.964175,Restaurant Ask,60.172366,24.955864,Scandinavian Restaurant
13,Kruununhaka,60.172643,24.964175,Ravintola Nokka,60.169144,24.961199,Restaurant


In [15]:
# Let's merge population data with venue list 
dfmerged.rename(index=str,columns={"name": "Neighborhood"},inplace=True)
venueswpopulation = pd.merge(dfmerged, venues, on="Neighborhood")
venueswpopulation.head()

,Neighborhood,lat,loc,2019,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kruununhaka,60.172643,24.964175,7470,60.172643,24.964175,Ravintola Kolme Kruunua,60.174297,24.958925,Scandinavian Restaurant
1,Kruununhaka,60.172643,24.964175,7470,60.172643,24.964175,Kuurna,60.170128,24.958564,Scandinavian Restaurant
2,Kruununhaka,60.172643,24.964175,7470,60.172643,24.964175,Korea House,60.172910,24.956436,Korean Restaurant
3,Kruununhaka,60.172643,24.964175,7470,60.172643,24.964175,Restaurant Ask,60.172366,24.955864,Scandinavian Restaurant
4,Kruununhaka,60.172643,24.964175,7470,60.172643,24.964175,Ravintola Nokka,60.169144,24.961199,Restaurant


In [16]:
# Check a neigborhoods
venueswpopulation.loc[venueswpopulation['Neighborhood'] == 'Vallila']

,Neighborhood,lat,loc,2019,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
237,Vallila,60.195198,24.950665,10195,60.195198,24.950665,African Pots,60.197754,24.949462,African Restaurant
238,Vallila,60.195198,24.950665,10195,60.195198,24.950665,Brokadi,60.197531,24.950576,Chinese Restaurant
239,Vallila,60.195198,24.950665,10195,60.195198,24.950665,KOKORO Sushi,60.193852,24.950727,Sushi Restaurant
240,Vallila,60.195198,24.950665,10195,60.195198,24.950665,King Kebab,60.194911,24.956125,Kebab Restaurant
241,Vallila,60.195198,24.950665,10195,60.195198,24.950665,Lebanese Food,60.197179,24.950662,Middle Eastern Restaurant
242,Vallila,60.195198,24.950665,10195,60.195198,24.950665,Mashiro,60.193697,24.958275,Japanese Restaurant
243,Vallila,60.195198,24.950665,10195,60.195198,24.950665,Restaurante Coco Grill,60.194548,24.957480,Venezuelan Restaurant
244,Vallila,60.195198,24.950665,10195,60.195198,24.950665,Alice Italian,60.191283,24.949234,Italian Restaurant
245,Vallila,60.195198,24.950665,10195,60.195198,24.950665,Thai Orchid-konepaja,60.193210,24.943615,Thai Restaurant


In [17]:
#Number of restaurants per area
ven = venueswpopulation.groupby('Neighborhood', as_index=False).count()

In [18]:
#Population per area
poplist = venueswpopulation[['Neighborhood','2019','lat','loc']]
poplist = poplist.drop_duplicates()
poplist.head()

,Neighborhood,2019,lat,loc
0,Kruununhaka,7470,60.172643,24.964175
16,Kluuvi,649,60.173882,24.942115
23,Kaartinkaupunki,1056,60.165507,24.950692
51,Punavuori,9417,60.161397,24.935337
82,Eira,1108,60.155592,24.939001


In [19]:
#Let's combine these and calculate how many restaurants there are per capita in each neighborhood
poplista = pd.merge(ven, poplist, on="Neighborhood")
poplista.rename(index=str,columns={"2019_x": "Restaurants", "2019_y":"Population","lat_y":"lat", "loc_y":"loc"},inplace=True)
poplista[["Restaurants", "Population"]] = poplista[["Restaurants", "Population"]].apply(pd.to_numeric)
poplista['Rest per 1000 capita'] = (poplista['Restaurants'] / poplista['Population'])*1000
poplista=poplista[["Neighborhood","Restaurants","Population","Rest per 1000 capita","lat","loc"]]
poplista


,Neighborhood,Restaurants,Population,Rest per 1000 capita,lat,loc
0,Ala-Malmi,1,6481,0.154297,60.244835,25.017064
1,Alppila,6,4602,1.303781,60.190469,24.941519
2,Arabianranta,1,7550,0.132450,60.205128,24.978939
3,Aurinkolahti,3,8003,0.374859,60.202623,25.155513
4,Eestinmalmi,1,3145,0.317965,60.165339,24.695261
5,Eira,9,1108,8.122744,60.155592,24.939001
6,Etelä-Haaga,1,12488,0.080077,60.212566,24.889656
7,Etelä-Leppävaara,7,7425,0.942761,60.213938,24.806510
8,Harju,14,7566,1.850383,60.188088,24.954459
9,Haukilahti,2,5677,0.352299,60.158607,24.776324


In [20]:
# How many neighborhoods are there with at least one restaurant found
poplista.shape[0]

88

In [21]:
poplista["Rest per 1000 capita"].min()

0.014351732254083068

In [22]:
# Function to show map and draw markers 
# use different colors based on restaurant density per capita
def showMap(latitude, longitude, neighborhood_data):
    
    map = folium.Map(location=[latitude, longitude], zoom_start=10)

    # add markers to map
    for lat, lng, neighborhood, density in zip(neighborhood_data['lat'], neighborhood_data['loc'], neighborhood_data['Neighborhood'], neighborhood_data['Rest per 1000 capita']):
        label = '{}'.format(neighborhood)
        label = folium.Popup(label, parse_html=True)
        if density < 0.1: c="white"
        elif density < 0.3: c="grey"
        elif density < 0.6: c="blue"
        elif density < 1.0: c="black"
        else :c="red"
            
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=c,
            fill=True,
            fill_color=c,
            fill_opacity=0.7,
            parse_html=False).add_to(map)  
    
    return map

In [23]:
#Let's draw a map with color coded neighbourhoods where restaurants are found
map = showMap(60.192059, 24.945831,poplista)
map

Let's then do a bit of clustering with the neighborhoods. The method used is K-means clustering.

In [24]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
# and change it to position 1
onehot['Neighborhood'] = venues['Neighborhood'] 
cols = onehot.columns.tolist()
cols = cols[-1:] + cols[:-1]
onehot = onehot[cols]

onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Asian Restaurant,Cambodian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Dumpling Restaurant,Falafel Restaurant,...,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Szechuan Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
2,Kruununhaka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Kruununhaka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Kruununhaka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Kruununhaka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,Kruununhaka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
grouped_venues = onehot.groupby('Neighborhood').mean().reset_index()
grouped_venues.head()

,Neighborhood,African Restaurant,American Restaurant,Asian Restaurant,Cambodian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Dumpling Restaurant,Falafel Restaurant,...,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Szechuan Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,Ala-Malmi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,0.000000
1,Alppila,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.166667,0.0,0.166667,0.0,0.0,0.0,0.0,0.166667
2,Arabianranta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
3,Aurinkolahti,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.333333,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,Eestinmalmi,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
import numpy as np # library to handle data in a vectorized manner

In [28]:
#Let's figure out the most popular types of restaurant per area
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped_venues['Neighborhood']

for ind in np.arange(grouped_venues.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped_venues.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ala-Malmi,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
1,Alppila,Greek Restaurant,Vietnamese Restaurant,Thai Restaurant,Italian Restaurant,Sushi Restaurant,French Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Indonesian Restaurant
2,Arabianranta,Himalayan Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
3,Aurinkolahti,Sri Lankan Restaurant,Indian Restaurant,Himalayan Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant
4,Eestinmalmi,Asian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
5,Eira,Italian Restaurant,Scandinavian Restaurant,Restaurant,Turkish Restaurant,Mexican Restaurant,Sushi Restaurant,Modern European Restaurant,French Restaurant,Japanese Restaurant,Indonesian Restaurant
6,Etelä-Haaga,Chinese Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
7,Etelä-Leppävaara,Mediterranean Restaurant,Thai Restaurant,Japanese Restaurant,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Fast Food Restaurant,Vietnamese Restaurant,German Restaurant,Korean Restaurant
8,Harju,Vietnamese Restaurant,Chinese Restaurant,Greek Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Italian Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Doner Restaurant,Indonesian Restaurant
9,Haukilahti,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant


In [29]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [30]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped_venues.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 3, 0, 2, 0, 3, 0, 0, 0,
       0, 1, 0, 0, 0, 2, 0, 2, 4, 2, 0, 1, 2, 0, 1, 0, 2], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged_data = poplista

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged_data = merged_data.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how = 'inner')

merged_data.head()

,Neighborhood,Restaurants,Population,Rest per 1000 capita,lat,loc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ala-Malmi,1,6481,0.154297,60.244835,25.017064,0,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
1,Alppila,6,4602,1.303781,60.190469,24.941519,0,Greek Restaurant,Vietnamese Restaurant,Thai Restaurant,Italian Restaurant,Sushi Restaurant,French Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Indonesian Restaurant
2,Arabianranta,1,7550,0.132450,60.205128,24.978939,0,Himalayan Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
3,Aurinkolahti,3,8003,0.374859,60.202623,25.155513,0,Sri Lankan Restaurant,Indian Restaurant,Himalayan Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant
4,Eestinmalmi,1,3145,0.317965,60.165339,24.695261,0,Asian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant


Create map with different color-coded clusters

In [32]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [33]:
# create map
map_clusters = folium.Map(location=[60.192059, 24.945831], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_data['lat'], merged_data['loc'], merged_data['Neighborhood'], merged_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Which neighborhoods belong to each of the five clusters?

In [34]:
# Cluster zero 
merged_data.loc[merged_data['Cluster Labels'] == 0, merged_data.columns[[0] + list(range(7, merged_data.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ala-Malmi,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
1,Alppila,Greek Restaurant,Vietnamese Restaurant,Thai Restaurant,Italian Restaurant,Sushi Restaurant,French Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Indonesian Restaurant
2,Arabianranta,Himalayan Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
3,Aurinkolahti,Sri Lankan Restaurant,Indian Restaurant,Himalayan Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant
4,Eestinmalmi,Asian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
5,Eira,Italian Restaurant,Scandinavian Restaurant,Restaurant,Turkish Restaurant,Mexican Restaurant,Sushi Restaurant,Modern European Restaurant,French Restaurant,Japanese Restaurant,Indonesian Restaurant
6,Etelä-Haaga,Chinese Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
7,Etelä-Leppävaara,Mediterranean Restaurant,Thai Restaurant,Japanese Restaurant,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Fast Food Restaurant,Vietnamese Restaurant,German Restaurant,Korean Restaurant
8,Harju,Vietnamese Restaurant,Chinese Restaurant,Greek Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Italian Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Doner Restaurant,Indonesian Restaurant
10,Herttoniemenranta,Asian Restaurant,Mediterranean Restaurant,Spanish Restaurant,Greek Restaurant,Restaurant,Vietnamese Restaurant,German Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant


In [35]:
# Cluster one
merged_data.loc[merged_data['Cluster Labels'] == 1, merged_data.columns[[0] + list(range(7, merged_data.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Haukilahti,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
14,Kaitaa,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
24,Koivuhaka,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
34,Lapinlahti,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
37,Luukki,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
40,Malminkartano,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
46,Nuijala,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
53,Pohjois-Laajalahti,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
56,Puistola,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
61,Roihupelto,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant


In [36]:
# Cluster two 
merged_data.loc[merged_data['Cluster Labels'] == 2, merged_data.columns[[0] + list(range(7, merged_data.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Kanta-Leppävaara,Indian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Himalayan Restaurant
28,Kuitinmäki,Indian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Himalayan Restaurant
30,Kuurinniitty,Indian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Himalayan Restaurant
32,Laajalahti,Indian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Himalayan Restaurant
35,Lassila,Indian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Himalayan Restaurant
39,Länsi-Herttoniemi,Indian Restaurant,Scandinavian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant
45,Niemenmäki,Sushi Restaurant,Indian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant
84,Viertola,Indian Restaurant,Restaurant,Vietnamese Restaurant,Lebanese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant


In [37]:
# Cluster three
merged_data.loc[merged_data['Cluster Labels'] == 3, merged_data.columns[[0] + list(range(7, merged_data.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Kaivoksela,Fast Food Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
19,Karvasmäki,Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
43,Muurala,Fast Food Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
48,Otaniemi,Fast Food Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
51,Pakkala,Fast Food Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
52,Patola,Chinese Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Greek Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant
57,Pukinmäki,Fast Food Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
78,Vaarala,Fast Food Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant


In [38]:
# Cluster four
merged_data.loc[merged_data['Cluster Labels'] == 4, merged_data.columns[[0] + list(range(7, merged_data.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Käpylä,Thai Restaurant,Scandinavian Restaurant,Vietnamese Restaurant,French Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
41,Meri-Rastila,Scandinavian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
47,Nuottaniemi,Scandinavian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
75,Tullisaari,Scandinavian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
80,Vanha Munkkiniemi,Scandinavian Restaurant,Vietnamese Restaurant,German Restaurant,Kurdish Restaurant,Korean Restaurant,Kebab Restaurant,Japanese Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant
